# End-to-End Pipeline & HTML Dashboard Explanation

This notebook documents the full flow of the assignment:

1. **Data ingestion** from Kaggle via Meltano (tap-csv) into BigQuery
2. **Modelling** with dbt (star schema across orders, customers, items, payments, reviews)
3. **Data quality** checks with Great Expectations (GX)
4. **Machine Learning** (XGBoost regressor predicting `freight_value`)
5. **Business & sentiment analytics** using a fact table with sentiment labels
6. **HTML dashboard generation** for CEO/COO and technical teams

The goal is to show that the group has implemented a **modern data pipeline** and can turn it into
business-ready insights and visualisations.

## 1. Pipeline Architecture (High-Level)

The pipeline we built can be summarised as:

```text
Kaggle (Brazilian E-Commerce CSVs)
   ↓  (download_kaggle.py / kagglehub / CSV export)
Meltano (tap-csv → target-bigquery)
   ↓
BigQuery raw tables (staging layer)
   ↓
dbt models (stg_*, dim_*, fact_*) in star schema
   ↓
dbt tests + Great Expectations suite (data quality gates)
   ↓
ML notebook (XGBoost: predict freight_value, evaluate RMSE/MAE/R²)
   ↓
Sentiment fact table + HTML dashboard
```

In the group presentation, this can be shown as a single slide with the tools logos
and arrows, and this notebook provides the technical explanation.

## 2. Load Modelled Fact Table with Sentiment

We use a prepared CSV exported from the modelled layer:

- One row per `order_id`
- Includes `payment_value` (revenue), `freight_value`, `customer_state`
- Includes `review_score` and a derived `sentiment_label`

This is the same file that powers the HTML dashboard.

In [ ]:
import pandas as pd
from pathlib import Path

BASE_DIR = Path('/home/pingh/Brazilian-E-Commerce-Public-Dataset-by-Olist')
DATA_PATH = BASE_DIR / 'dashboard' / 'fact_orders_with_sentiment.csv'

df = pd.read_csv(DATA_PATH)
print('Rows:', len(df))
df.head()

### 2.1. Check Columns and Sentiment Logic

We verify that the core columns exist and that sentiment is consistent with review score:
- 4–5 → positive
- 3 → neutral
- 1–2 → negative

In [ ]:
df.columns.tolist()

In [ ]:
df[['review_score', 'sentiment_label']].value_counts().sort_index()

## 3. Business KPIs Derived from the Fact Table

These KPIs are used in both the Jupyter dashboards and the HTML dashboard:

- Total revenue
- Total orders
- Average order value
- Average freight cost
- Average review score
- Sentiment distribution and percentage of positive / negative reviews

This section can be copy-pasted into the written report as a numerical summary.

In [ ]:
total_revenue = float(df['payment_value'].sum())
total_orders = int(df['order_id'].nunique())
avg_order_value = float(df['payment_value'].mean())
avg_freight = float(df['freight_value'].mean())
avg_review = float(df['review_score'].mean())

sent_counts = df['sentiment_label'].value_counts()
sent_pct = (sent_counts / sent_counts.sum() * 100).round(1)

summary = pd.DataFrame({
    'metric': [
        'Total Revenue (BRL)',
        'Total Orders',
        'Average Order Value (BRL)',
        'Average Freight Value (BRL)',
        'Average Review Score (1–5)',
        '% Positive Sentiment',
        '% Neutral Sentiment',
        '% Negative Sentiment',
    ],
    'value': [
        f"{total_revenue:,.0f}",
        f"{total_orders:,}",
        f"{avg_order_value:,.2f}",
        f"{avg_freight:,.2f}",
        f"{avg_review:,.2f}",
        f"{sent_pct.get('positive', 0.0):.1f}%",
        f"{sent_pct.get('neutral', 0.0):.1f}%",
        f"{sent_pct.get('negative', 0.0):.1f}%",
    ]
})

summary

## 4. Machine Learning Model (XGBoost Regressor)

The ML component uses an **XGBoost Regressor** to predict `freight_value`.
This is useful because freight cost is a key driver of customer dissatisfaction and margin.

From the ML notebook, we obtained the following performance on the test set:

- **RMSE**: 9.36
- **MAE**: 3.90
- **R²**: 0.6334

Interpretation:

- RMSE ≈ 9.36 means that, on average, the model's prediction is within ~9.36 BRL of the true freight cost.
- MAE ≈ 3.90 confirms that most errors are relatively small compared to the range of freight values.
- R² ≈ 0.63 indicates that the model explains around 63% of the variance in `freight_value`,
  which is reasonable given the noisy nature of logistics data.

These metrics are later displayed in the HTML dashboard header as part of the **"ML & Business Insights"** story.

In [ ]:
XGB_RMSE = 9.36
XGB_MAE = 3.90
XGB_R2 = 0.6334

pd.DataFrame({
    'metric': ['RMSE', 'MAE', 'R2'],
    'value': [XGB_RMSE, XGB_MAE, XGB_R2]
})

## 5. Quick Visuals Used in the Dashboard

The HTML dashboard uses Plotly charts embedded into an HTML template.
Here we preview a couple of key visuals directly in the notebook:

- Monthly revenue trend
- Sentiment share over time
- Freight cost vs review score

These serve as a sanity check that the data feeding the HTML dashboard is correct.

In [ ]:
import plotly.express as px
import pandas as pd

df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['year_month'] = df['order_purchase_timestamp'].dt.to_period('M').astype(str)

monthly = df.groupby('year_month').agg(
    revenue=('payment_value', 'sum'),
    orders=('order_id', 'nunique')
).reset_index()

fig_rev = px.line(
    monthly,
    x='year_month',
    y='revenue',
    title='Monthly Revenue Trend',
    labels={'year_month': 'Month', 'revenue': 'Revenue (BRL)'}
)
fig_rev.show()

In [ ]:
sent_time = (
    df.groupby(['year_month', 'sentiment_label'])
    .size()
    .reset_index(name='count')
)
sent_time['share'] = sent_time.groupby('year_month')['count'].transform(lambda x: x / x.sum())

fig_sent = px.area(
    sent_time,
    x='year_month',
    y='share',
    color='sentiment_label',
    title='Sentiment Share Over Time',
    labels={'year_month': 'Month', 'share': 'Share of Reviews'}
)
fig_sent.show()

In [ ]:
fig_box = px.box(
    df,
    x='review_score',
    y='freight_value',
    points='outliers',
    title='Freight Cost vs Review Score',
    labels={'review_score': 'Review Score', 'freight_value': 'Freight Value'}
)
fig_box.show()

## 6. HTML Dashboard Generation

The HTML dashboard is generated by a separate Python script (for example `generate_html_dashboard.py`).
That script:

1. Loads `fact_orders_with_sentiment.csv`
2. Recomputes KPIs (revenue, orders, sentiment ratios)
3. Builds Plotly charts (revenue trend, sentiment over time, revenue by state, sentiment by state, freight vs score)
4. Embeds the charts and KPIs into a custom HTML template with a dark, executive-style layout
5. Writes out a standalone HTML file, e.g. `dashboard/olist_ml_business_dashboard.html`

In the assignment, you can reference this notebook as the **technical design** and
the HTML file as the **final business-facing artefact**.

In [ ]:
html_path = BASE_DIR / 'dashboard' / 'olist_ml_business_dashboard.html'
print('If you have already run generate_html_dashboard.py, the dashboard should be here:')
print(html_path)
print('You can open it in a browser or attach screenshots in the slide deck.')

## 7. How to Present This in the Assignment

Suggested structure for your presentation:

1. **Problem & Dataset**  
   Brazilian e-commerce marketplace, need to understand revenue, freight cost, and customer satisfaction.

2. **Architecture**  
   Show the end-to-end diagram: Kaggle → Meltano → BigQuery → dbt → GX → ML → Dashboard.

3. **Data Modelling & Quality**  
   Explain the star schema and a few key dbt/GX tests (e.g. no nulls in primary keys, valid ranges, referential integrity).

4. **ML Model**  
   Briefly explain features, XGBoost choice, and the evaluation metrics (RMSE/MAE/R²).

5. **Dashboard Demo**  
   Open the HTML dashboard, highlight the KPIs and key charts, and link them back to the business questions.

6. **Insights & Recommendations**  
   Summarise what the data shows (e.g. which states are risky, how freight affects satisfaction)
   and propose 2–3 concrete actions.

This notebook can be submitted as technical evidence that the team understands and can reproduce the full pipeline.